# Notebook used to apply standard compression methods to Kodak dataset image for evaluation
## Importing all the necessary packages and initializing input and output directories

In [228]:
import os
import ntpath
from PIL import Image
from pathlib import Path
import glob
import timeit
import subprocess
from numpy import mean
from datetime import datetime

vvenc_path = Path('/home/filippo/Downloads/vvenc-1.9.1/install/bin/vvencapp').resolve()
print ("VVEncAPP Path: ", str(vvenc_path))
vvdec_path = Path('/home/filippo/Downloads/vvdec-2.1.2/install/bin/vvdecapp').resolve()
print ("VVDevAPP Path: ", str(vvdec_path))

current_path = Path('.').resolve()
print("Current path: ",str(current_path))
source_path = str(current_path)+'/IMAGES/PNG_IMAGES'
source_path.replace(os.sep,ntpath.sep)
print("Source: ",source_path)

VVEncAPP Path:  /home/filippo/Downloads/vvenc-1.9.1/install/bin/vvencapp
VVDevAPP Path:  /home/filippo/Downloads/vvdec-2.1.2/install/bin/vvdecapp
Current path:  /home/filippo/DataspellProjects/ImageCompressionAI
Source:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/PNG_IMAGES


# JPEG conversion
## Setting output dir for JPEG file conversion

In [229]:
destination_path = str(current_path)+'/IMAGES/JPEG_IMAGES'
destination_path.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

destination_path_JPEG = str(current_path)+'/IMAGES/JPEG_IMAGES/JPEG'
destination_path_JPEG.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path_JPEG)
if os.path.isdir(destination_path_JPEG) & os.path.exists(destination_path_JPEG):
    try:
        files = os.listdir(destination_path_JPEG)
        for file in files:
            file_path = os.path.join(destination_path_JPEG, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path_JPEG)

Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG_IMAGES
Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG_IMAGES/JPEG


## Converting images into JPEG format

In [230]:
compression_times = []
quality_index = 11 # The image quality, on a scale from 0 (worst) to 95 (best)
for file in glob.glob(str(source_path)+'/*.png'):
    image = Image.open(file)
    image = image.convert('RGB')
    starttime = timeit.default_timer()
    image.save(str(destination_path_JPEG)+'/'+Path(file).stem+'.jpeg','jpeg',subsampling=1, quality=quality_index)
    endtime = timeit.default_timer()
    execution_time = endtime-starttime
    compression_times.append(execution_time)
    print('Encoded',Path(file).stem,' | Execution time: ',str(execution_time),' seconds')
    image.close()

Encoded IMG0001  | Execution time:  0.014038969999091933  seconds
Encoded IMG0002  | Execution time:  0.013927517000411171  seconds
Encoded IMG0003  | Execution time:  0.013317149998329114  seconds
Encoded IMG0004  | Execution time:  0.01350164500036044  seconds
Encoded IMG0005  | Execution time:  0.014457010998739861  seconds
Encoded IMG0006  | Execution time:  0.013691281001229072  seconds
Encoded IMG0007  | Execution time:  0.013455704000080004  seconds
Encoded IMG0008  | Execution time:  0.013989019000291592  seconds
Encoded IMG0009  | Execution time:  0.013141015002474887  seconds
Encoded IMG0010  | Execution time:  0.013142316001903964  seconds
Encoded IMG0011  | Execution time:  0.013783132999378722  seconds
Encoded IMG0012  | Execution time:  0.013080524000542937  seconds
Encoded IMG0013  | Execution time:  0.014337167998746736  seconds
Encoded IMG0014  | Execution time:  0.013795523002045229  seconds
Encoded IMG0015  | Execution time:  0.013319010999111924  seconds
Encoded IMG

## Converting JPEG compressed images back to PNG for later comparison

In [232]:
for file in glob.glob(str(destination_path_JPEG)+'/*.jpeg'):
    image = Image.open(file)
    image = image.convert('RGB')
    image.save(str(destination_path)+'/'+Path(file).stem+'.png','png')
    print(Path(file).stem)
    image.close()

IMG0001
IMG0002
IMG0003
IMG0004
IMG0005
IMG0006
IMG0007
IMG0008
IMG0009
IMG0010
IMG0011
IMG0012
IMG0013
IMG0014
IMG0015
IMG0016
IMG0017
IMG0018
IMG0019
IMG0020
IMG0021
IMG0022
IMG0023
IMG0024


## Compute bits per pixel of converted images

In [231]:
bpp_list=[]
count = 0
log = open(str(destination_path)+'/log.txt', 'a')
log.writelines([str(datetime.now()),'\n'])
for file in glob.glob(str(destination_path_JPEG)+'/*.jpeg'):
    image = Image.open(file)
    file_size = os.path.getsize(file) * 8
    pixels = image.width * image.height
    bits_per_pixel = file_size/pixels
    bpp_list.append(bits_per_pixel)
    log_string = 'Image:' + str(Path(file).stem) + '.jpeg\tbpp:' + str(bits_per_pixel) + '\tSize:' + str(file_size) + '\tWidth:' + str(image.width) + '\tHeight:' + str(image.height) + '\tPixels:' + str(pixels) + '\tCompressionTime:' + str(compression_times[count])
    log.writelines([log_string,'\n'])
    print(log_string)
    image.close()
    count+=1
    
log.writelines(['\n',str('Average bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')])    
print('\nAverage bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')
log.close()

Image:IMG0001.jpeg	bpp:0.31909942626953125	Size:2007600	Width:3072	Height:2048	Pixels:6291456	CompressionTime:0.014038969999091933
Image:IMG0002.jpeg	bpp:0.25029881795247394	Size:1574744	Width:3072	Height:2048	Pixels:6291456	CompressionTime:0.013927517000411171
Image:IMG0003.jpeg	bpp:0.19769922892252603	Size:1243816	Width:3072	Height:2048	Pixels:6291456	CompressionTime:0.013317149998329114
Image:IMG0004.jpeg	bpp:0.23328145345052084	Size:1467680	Width:2048	Height:3072	Pixels:6291456	CompressionTime:0.01350164500036044
Image:IMG0005.jpeg	bpp:0.3073323567708333	Size:1933568	Width:3072	Height:2048	Pixels:6291456	CompressionTime:0.014457010998739861
Image:IMG0006.jpeg	bpp:0.26929601033528644	Size:1694264	Width:3072	Height:2048	Pixels:6291456	CompressionTime:0.013691281001229072
Image:IMG0007.jpeg	bpp:0.21540451049804688	Size:1355208	Width:3072	Height:2048	Pixels:6291456	CompressionTime:0.013455704000080004
Image:IMG0008.jpeg	bpp:0.3476676940917969	Size:2187336	Width:3072	Height:2048	Pixels:

# JPEG2000 Conversion
## Setting output directory for JPEG2000 file conversion

In [233]:
destination_path = str(current_path)+'/IMAGES/JPEG2000_IMAGES'
destination_path.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

destination_path_JPEG2000 = str(current_path)+'/IMAGES/JPEG2000_IMAGES/JPEG2000'
destination_path_JPEG2000.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path_JPEG2000)
if os.path.isdir(destination_path_JPEG2000) & os.path.exists(destination_path_JPEG2000):
    try:
        files = os.listdir(destination_path_JPEG2000)
        for file in files:
            file_path = os.path.join(destination_path_JPEG2000, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path_JPEG2000)

Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES/JPEG2000


## Converting Images to JPEG2000

In [234]:
compression_times = []
quality_layers = [95]
for file in glob.glob(str(source_path)+'/*.png'):
    image = Image.open(file)
    image = image.convert('RGB')
    starttime = timeit.default_timer()
    image.save(str(destination_path_JPEG2000)+'/'+Path(file).stem+'.jp2','jpeg2000',irreversible=True, quality_mode='rates',quality_layers=quality_layers)
    endtime = timeit.default_timer()
    execution_time = endtime-starttime
    compression_times.append(execution_time)
    print('Encoded',Path(file).stem,' | Execution time: ',execution_time,' seconds')
    image.close()

Encoded IMG0001  | Execution time:  2.6986739830026636  seconds
Encoded IMG0002  | Execution time:  2.2322539760025393  seconds
Encoded IMG0003  | Execution time:  2.072450070998457  seconds
Encoded IMG0004  | Execution time:  2.148673793999478  seconds
Encoded IMG0005  | Execution time:  2.2333629459972144  seconds
Encoded IMG0006  | Execution time:  2.1902307640011713  seconds
Encoded IMG0007  | Execution time:  2.229076409999834  seconds
Encoded IMG0008  | Execution time:  3.0746533340025053  seconds
Encoded IMG0009  | Execution time:  2.798849323000468  seconds
Encoded IMG0010  | Execution time:  2.73212425499878  seconds
Encoded IMG0011  | Execution time:  2.379197516002023  seconds
Encoded IMG0012  | Execution time:  1.9838918940004078  seconds
Encoded IMG0013  | Execution time:  2.7664589090018126  seconds
Encoded IMG0014  | Execution time:  2.2733290340001986  seconds
Encoded IMG0015  | Execution time:  1.903778265001165  seconds
Encoded IMG0016  | Execution time:  2.2077255050

## Converting JPEG200 compressed images back to PNG for later comparison

In [239]:
for file in glob.glob(str(destination_path_JPEG2000)+'/*.jp2'):
    image = Image.open(file)
    image = image.convert('RGB')
    image.save(str(destination_path)+'/'+Path(file).stem+'.png','png')
    print(Path(file).stem)
    image.close()

/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
IMG0001
/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
IMG0002
/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
IMG0003
/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
IMG0004
/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
IMG0005
/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
IMG0006
/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
IMG0007
/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
IMG0008
/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
IMG0009
/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
IMG0010
/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
IMG0011
/home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/JPEG2000_IMAGES
IMG0012
/home/filippo/Da

## Compute bits per pixel of converted images

In [237]:
bpp_list=[]
count = 0
log = open(str(destination_path)+'/log.txt', 'a')
log.writelines([str(datetime.now()),'\n'])
for file in glob.glob(str(destination_path_JPEG2000)+'/*.jp2'):
    image = Image.open(file)
    file_size = os.path.getsize(file) * 8
    pixels = image.width * image.height
    bits_per_pixel = file_size/pixels
    bpp_list.append(bits_per_pixel)
    log_string = 'Image:' + str(Path(file).stem) + '.jp2\tbpp:' + str(bits_per_pixel) + '\tSize:' + str(file_size) + '\tWidth:' + str(image.width) + '\tHeight:' + str(image.height) + '\tPixels:' + str(pixels) + '\tCompressionTime:' + str(compression_times[count])
    log.writelines([log_string,'\n'])
    print(log_string)
    image.close()
    count +=1

log.writelines(['\n',str('Average bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')])
print('\nAverage bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')
log.close()

Image:IMG0001.jp2	bpp:0.25246938069661456	Size:1588400	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.6986739830026636
Image:IMG0002.jp2	bpp:0.25247955322265625	Size:1588464	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.2322539760025393
Image:IMG0003.jp2	bpp:0.2525749206542969	Size:1589064	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.072450070998457
Image:IMG0004.jp2	bpp:0.2524223327636719	Size:1588104	Width:2048	Height:3072	Pixels:6291456	CompressionTime:2.148673793999478
Image:IMG0005.jp2	bpp:0.2524668375651042	Size:1588384	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.2333629459972144
Image:IMG0006.jp2	bpp:0.2524833679199219	Size:1588488	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.1902307640011713
Image:IMG0007.jp2	bpp:0.2525927225748698	Size:1589176	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.229076409999834
Image:IMG0008.jp2	bpp:0.2526105244954427	Size:1589288	Width:3072	Height:2048	Pixels:6291456	CompressionTime:3.07

# BPG Conversion
## Setting output directory for BPG conversion

In [240]:
destination_path = str(current_path)+'/IMAGES/BPG_IMAGES'
destination_path.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

destination_path_BPG = str(current_path)+'/IMAGES/BPG_IMAGES/BPG'
destination_path_BPG.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path_BPG)
if os.path.isdir(destination_path_BPG) & os.path.exists(destination_path_BPG):
    try:
        files = os.listdir(destination_path_BPG)
        for file in files:
            file_path = os.path.join(destination_path_BPG, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path_BPG)

Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/BPG_IMAGES
Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/BPG_IMAGES/BPG


## Converting Images to BPG

In [242]:
execution_times = []
quantizer_parameter = 40 #set quantizer parameter (smaller gives better quality,range: 0-51, default = 29)
for file in os.listdir(source_path):
    source_file = str(source_path + '/' + file)
    source_file.replace(os.sep,ntpath.sep)
    destination_file = str(destination_path_BPG + '/' + str(Path(file).stem) + '.bpg')
    destination_file.replace(os.sep,ntpath.sep)
    starttime = timeit.default_timer()
    subprocess.run(['bpgenc','-c','rgb','-f','420','-m','1','-q',str(quantizer_parameter),'-o',str(destination_file),str(source_file)])
    endtime = timeit.default_timer()
    execution_time = endtime - starttime
    execution_times.append(execution_time)
    print('Encoded', Path(file).stem, ' | Execution time: ', str(execution_time), ' seconds')

Encoded IMG0001  | Execution time:  1.0622869329999958  seconds
Encoded IMG0002  | Execution time:  0.5575756440011901  seconds
Encoded IMG0003  | Execution time:  0.5034589350034366  seconds
Encoded IMG0004  | Execution time:  0.5224798670024029  seconds
Encoded IMG0005  | Execution time:  0.5731400629993004  seconds
Encoded IMG0006  | Execution time:  0.5538719030009815  seconds
Encoded IMG0007  | Execution time:  0.5136101790012617  seconds
Encoded IMG0008  | Execution time:  0.6100415469991276  seconds
Encoded IMG0009  | Execution time:  0.4764426170004299  seconds
Encoded IMG0010  | Execution time:  0.4858262600027956  seconds
Encoded IMG0011  | Execution time:  0.5477884900028585  seconds
Encoded IMG0012  | Execution time:  0.4974130320006225  seconds
Encoded IMG0013  | Execution time:  0.6490037370022037  seconds
Encoded IMG0014  | Execution time:  0.559707640997658  seconds
Encoded IMG0015  | Execution time:  0.5154069780001009  seconds
Encoded IMG0016  | Execution time:  0.516

## Converting BPG compressed images back to PNG for later comparison

In [244]:
for file in os.listdir(destination_path_BPG):
    source_file = str(destination_path_BPG + '/' + file)
    source_file.replace(os.sep,ntpath.sep)
    destination_file = str(destination_path + '/' + str(Path(file).stem) + '.png')
    destination_file.replace(os.sep,ntpath.sep)
    subprocess.run(['bpgdec','-o',str(destination_file),str(source_file)])
    print(Path(file).stem)

IMG0001
IMG0002
IMG0003
IMG0004
IMG0005
IMG0006
IMG0007
IMG0008
IMG0009
IMG0010
IMG0011
IMG0012
IMG0013
IMG0014
IMG0015
IMG0016
IMG0017
IMG0018
IMG0019
IMG0020
IMG0021
IMG0022
IMG0023
IMG0024


## Compute bits per pixel of converted images

In [243]:
bpp_list=[]
count = 0
log = open(str(destination_path)+'/log.txt', 'a')
log.writelines([str(datetime.now()),'\n'])
for file in os.listdir(destination_path_BPG):
    filepath = str(destination_path_BPG) + '/' + str(file)
    filepath.replace(os.sep,ntpath.sep)
    file_size = os.path.getsize(filepath) * 8
    output = subprocess.Popen(['bpgdec','-i',str(filepath)],stdout=subprocess.PIPE).communicate()
    dimensions = (((str(output).split(' '))[0].split('='))[1].split('x'))
    pixels = int(dimensions[0]) * int(dimensions [1])
    bits_per_pixel = file_size/pixels
    bpp_list.append(bits_per_pixel)
    log_string = 'Image:' + str(Path(file).stem) + '.bpg\tbpp:' + str(bits_per_pixel) + '\tSize:' + str(file_size) + '\tWidth:' + str(dimensions[0]) + '\tHeight:' + str(dimensions[1]) + '\tPixels:' + str(pixels) + '\tCompressionTime:' + str(compression_times[count])
    log.writelines([log_string,'\n'])
    print(log_string)
    count +=1

log.writelines(['\n',str('Average bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')])
print('\nAverage bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')
log.close()

Image:IMG0001.bpg	bpp:0.4085502624511719	Size:2570376	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.6986739830026636
Image:IMG0002.bpg	bpp:0.2327295939127604	Size:1464208	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.2322539760025393
Image:IMG0003.bpg	bpp:0.09084192911783855	Size:571528	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.072450070998457
Image:IMG0004.bpg	bpp:0.182220458984375	Size:1146432	Width:2048	Height:3072	Pixels:6291456	CompressionTime:2.148673793999478
Image:IMG0005.bpg	bpp:0.3009847005208333	Size:1893632	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.2333629459972144
Image:IMG0006.bpg	bpp:0.2892405192057292	Size:1819744	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.1902307640011713
Image:IMG0007.bpg	bpp:0.10905710856119792	Size:686128	Width:3072	Height:2048	Pixels:6291456	CompressionTime:2.229076409999834
Image:IMG0008.bpg	bpp:0.4129778544108073	Size:2598232	Width:3072	Height:2048	Pixels:6291456	CompressionTime:3.07465

# VVC Conversion
## Setting input and output directory for VVC conversion

In [246]:
source_path = str(current_path)+'/IMAGES/YUV_IMAGES'
source_path.replace(os.sep,ntpath.sep)
print("Source: ",source_path)

destination_path = str(current_path)+'/IMAGES/VVC_IMAGES'
destination_path.replace(os.sep,ntpath.sep)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

destination_path = str(current_path)+'/IMAGES/VVC_IMAGES/YUV'
destination_path.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

destination_path_VVC = str(current_path)+'/IMAGES/VVC_IMAGES/VVC'
destination_path_VVC.replace(os.sep,ntpath.sep)
print("Destination: ",destination_path_VVC)
if os.path.isdir(destination_path_VVC) & os.path.exists(destination_path_VVC):
    try:
        files = os.listdir(destination_path_VVC)
        for file in files:
            file_path = os.path.join(destination_path_VVC, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path_VVC)

log_path = str(current_path)+'/IMAGES/VVC_IMAGES'
source_path.replace(os.sep,ntpath.sep)
print("Log: ",source_path)

Source:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/YUV_IMAGES
Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC_IMAGES/YUV
Destination:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC_IMAGES/VVC
Log:  /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/YUV_IMAGES


## Converting images to VVC

In [251]:
compression_times = []
quantizer_parameter = 32  # 32 = 0.25bpp ;  Quantization parameter (0...63) 0=better quality
for file in os.listdir(source_path):
    if file.lower().endswith('.yuv'):
        dimensions = open(source_path+'/'+str(Path(file).stem)+'.txt').readline()
        source_file = str(source_path + '/' + file)
        source_file.replace(os.sep,ntpath.sep)
        destination_file = str(destination_path_VVC + '/' + str(Path(file).stem) + '.vvc')
        destination_file.replace(os.sep,ntpath.sep)
        starttime = timeit.default_timer()
        subprocess.run([str(vvenc_path),'--profile','main_10_still_picture','-f','1','-c','yuv420_10','-s',str(dimensions),'--qp',str(quantizer_parameter),'--input',str(source_file),'--output',str(destination_file)])
        endtime = timeit.default_timer()
        execution_time = endtime - starttime - 0.2
        compression_times.append(execution_time)
        print('Encoded', Path(file).stem, ' | Execution time: ', str(execution_time), ' seconds','\n\n')

vvencapp: Fraunhofer VVC Encoder ver. 1.9.1 [Linux][GCC 13.2.1][64 bit][SIMD=AVX2]
vvenc [info]: Input File                             : /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/YUV_IMAGES/IMG0001.yuv
vvenc [info]: Bitstream File                         : /home/filippo/DataspellProjects/ImageCompressionAI/IMAGES/VVC_IMAGES/VVC/IMG0001.vvc
vvenc [info]: Real Format                            : 3072x2048  yuv420p10  60 Hz  SDR  1 frame
vvenc [info]: Frames                                 : encode 1 frame 
vvenc [info]: Internal format                        : 3072x2048  60 Hz  SDR
vvenc [info]: Threads                                : 8  (parallel frames: 4)
vvenc [info]: Rate control                           : QP 32
vvenc [info]: Perceptual optimization                : Enabled
vvenc [info]: Intra period (keyframe)                : 64
vvenc [info]: Decoding refresh type                  : CRA


vvenc [info]: stats summary: frame= 1/1 avg_fps= 0.1 avg_bitrate= 125521.9

## Converting VVC Images back to YUV for later comparison
To convert images back to PNG refer to ImagePreparation

In [253]:
for file in os.listdir(destination_path_VVC):
    source_file = str(destination_path_VVC + '/' + file)
    source_file.replace(os.sep,ntpath.sep)
    destination_file = str(destination_path+ '/' + str(Path(file).stem) + '.yuv')
    destination_file.replace(os.sep,ntpath.sep)
    starttime = timeit.default_timer()
    subprocess.run([str(vvdec_path),'--bitstream',str(source_file),'--output',str(destination_file)])
    endtime = timeit.default_timer()
    print('Decoded', Path(file).stem, ' | Execution time: ', endtime - starttime, ' seconds','\n\n')

Fraunhofer VVC/H.266 Decoder VVdeC, version 2.1.2 [THREADS=12; PARSE_DELAY=12; SIMD=AVX2]
POC    0 LId:  0 TId: 0 ( I-SLICE, QP 29 ) [DT  0.142] [L0 ] [L1 ] 
vvdecapp [info]: SizeInfo: 3072x2048 (10b)
vvdecapp [info]: 2023-Oct-21 17:50:59.640788: 1 frames decoded @ 11.236 fps (0.089 sec)

Decoded IMG0001  | Execution time:  0.0994812289973197  seconds 


Fraunhofer VVC/H.266 Decoder VVdeC, version 2.1.2 [THREADS=12; PARSE_DELAY=12; SIMD=AVX2]
POC    0 LId:  0 TId: 0 ( I-SLICE, QP 27 ) [DT  0.167] [L0 ] [L1 ] 
vvdecapp [info]: SizeInfo: 3072x2048 (10b)
vvdecapp [info]: 2023-Oct-21 17:50:59.752993: 1 frames decoded @ 9.80392 fps (0.102 sec)

Decoded IMG0002  | Execution time:  0.11275190699961968  seconds 


Fraunhofer VVC/H.266 Decoder VVdeC, version 2.1.2 [THREADS=12; PARSE_DELAY=12; SIMD=AVX2]
POC    0 LId:  0 TId: 0 ( I-SLICE, QP 24 ) [DT  0.089] [L0 ] [L1 ] 
vvdecapp [info]: SizeInfo: 3072x2048 (10b)
vvdecapp [info]: 2023-Oct-21 17:50:59.824369: 1 frames decoded @ 16.3934 fps (0.061

## Compute bits per pixel of cf converted images

In [252]:
bpp_list=[]
count = 0
log = open(str(log_path)+'/log.txt', 'a')
log.writelines([str(datetime.now()),'\n'])
for file in os.listdir(destination_path_VVC):
    filepath = str(destination_path_VVC) + '/' + str(file)
    filepath.replace(os.sep,ntpath.sep)
    file_size = os.path.getsize(filepath) * 8
    dimensions = open(source_path+'/'+str(Path(file).stem)+'.txt').readline().split('x')
    pixels = int(dimensions[0]) * int(dimensions [1])
    bits_per_pixel = file_size/pixels
    bpp_list.append(bits_per_pixel)
    log_string = 'Image:' + str(Path(file).stem) + '.vvc\tbpp:' + str(bits_per_pixel) + '\tSize:' + str(file_size) + '\tWidth:' + str(dimensions[0]) + '\tHeight:' + str(dimensions[1]) + '\tPixels:' + str(pixels) + '\tCompressionTime:' + str(compression_times[count])
    log.writelines([log_string,'\n'])
    print(log_string)
    count +=1

log.writelines(['\n',str('Average bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')])
print('\nAverage bpp in ' + str(count) + ' images is ' + str(mean(bpp_list)) + ' in ' + str(mean(compression_times)) + ' seconds')
log.close()

Image:IMG0001.vvc	bpp:0.33251953125	Size:2092032	Width:3072	Height:2048	Pixels:6291456	CompressionTime:12.27677215099975
Image:IMG0002.vvc	bpp:0.3496233622233073	Size:2199640	Width:3072	Height:2048	Pixels:6291456	CompressionTime:15.155482715000108
Image:IMG0003.vvc	bpp:0.12453460693359375	Size:783504	Width:3072	Height:2048	Pixels:6291456	CompressionTime:9.73996007000169
Image:IMG0004.vvc	bpp:0.25571441650390625	Size:1608816	Width:2048	Height:3072	Pixels:6291456	CompressionTime:11.20087990200118
Image:IMG0005.vvc	bpp:0.26975123087565106	Size:1697128	Width:3072	Height:2048	Pixels:6291456	CompressionTime:12.228090903002158
Image:IMG0006.vvc	bpp:0.25035858154296875	Size:1575120	Width:3072	Height:2048	Pixels:6291456	CompressionTime:10.441205986001297
Image:IMG0007.vvc	bpp:0.13860448201497397	Size:872024	Width:3072	Height:2048	Pixels:6291456	CompressionTime:10.523656672998914
Image:IMG0008.vvc	bpp:0.34180450439453125	Size:2150448	Width:3072	Height:2048	Pixels:6291456	CompressionTime:18.00300